In [3]:
#data collection
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

#load the dataset

data = gutenberg.raw('shakespeare-hamlet.txt')

[nltk_data] Downloading package gutenberg to /Users/ADIL/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [4]:
#save the dataset to a file

with open('hamlet.txt', 'w') as f:
    f.write(data)

In [5]:
#data preprocessing

import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

#load the dataset

with open('hamlet.txt', 'r') as f:
    text = f.read().lower()

In [6]:
#tokenize the text creating indexes for words

tokenizer = Tokenizer()
tokenizer.fit_on_texts([text])
total_words = len(tokenizer.word_index)+1
total_words

4818

In [8]:
#create input sequences

input_sequences = []    #empty list
for line in text.split('\n'):     #split sentences from text into lines
    token_list = tokenizer.texts_to_sequences([line])[0]    #converting words to tokens(numbers)
    for i in range(1, len(token_list)):    #iterating on length of token_list
        n_gram_sequence = token_list[:i+1]   #building chunk of numbers called ngram ex: [1,2][1,2,3]
        input_sequences.append(n_gram_sequence)  #finally appending the n gram sequences to input_sequence

In [9]:
#pad sequences

max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))


In [10]:
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]], dtype=int32)

In [11]:
#create predictors and labels

x,y = input_sequences[:,:-1],input_sequences[:,-1]


In [12]:
#converting y values into categories ( eg: from 54,32 => 0,0,1,0, 0,0,0,1  one hot encoding)
import tensorflow as tf

y = tf.keras.utils.to_categorical(y, num_classes=total_words)
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [13]:
#splitting the data into train and test

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [16]:
#train LSTM RNN

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(LSTM(150,return_sequences = False))
model.add(Dropout(0.2))
model.add(Dense(total_words, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_2 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [17]:
history = model.fit(x_train, y_train, epochs=50,validation_data=(x_test,y_test), verbose=1)

Epoch 1/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 11s 17ms/step - accuracy: 0.0315 - loss: 7.0562 - val_accuracy: 0.0404 - val_loss: 6.6338
Epoch 2/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.0440 - loss: 6.4327 - val_accuracy: 0.0515 - val_loss: 6.6524
Epoch 3/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.0533 - loss: 6.2598 - val_accuracy: 0.0540 - val_loss: 6.6708
Epoch 4/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.0573 - loss: 6.0619 - val_accuracy: 0.0663 - val_loss: 6.6977
Epoch 5/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.0721 - loss: 5.8235 - val_accuracy: 0.0719 - val_loss: 6.7390
Epoch 6/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.0827 - loss: 5.6021 - val_accuracy: 0.0721 - val_loss: 6.8336
Epoch 7/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.0999 - loss: 5.3276 - val_accuracy: 0.0734 - val_loss: 6.9405
Epoch 8/50
644/644 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.1072 - loss: 5.0835 - va

In [18]:
#function to predict the next word

def predict_next_word(model, tokenizer, text, max_sequence_len):
  token_list = tokenizer.texts_to_sequences([text])[0]
  if len(token_list) >= max_sequence_len:
    token_list = token_list[-(max_sequence_len-1):]
  token_list = pad_sequences([token_list], maxlen= max_sequence_len-1, padding='pre')
  predicted = model.predict(token_list, verbose=0)
  predicted_word_index = np.argmax(predicted, axis=1)
  for word, index in tokenizer.word_index.items():
    if index == predicted_word_index:
      return word
  return None


In [20]:
#predicting next word

input_text = 'For this releefe'
max_sequence_len = model.input_shape[1]+1
predicted_word = predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f'input text : {input_text}')
print(f'predicted word : {predicted_word}')


input text : For this releefe
predicted word : much


In [21]:
#saving the model

model.save('lstm_next_word.h5')

#save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
  pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)